In [1]:
from collections import defaultdict
import gym
import numpy as np
import pandas as pd
import Configuration.config as cfg
from Evaluation.notebook_utils import *
from gym_atena.reactida.utils.utilities import Repository

%matplotlib inline


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pytz/tzinfo.py:27: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  _epoch = datetime.utcfromtimestamp(0)


Configuration loaded with:
  - humanity_coeff: 1.0
  - diversity_coeff: 2.0
  - kl_coeff: 1.5
  - compaction_coeff: 2.0
  - adam_lr: 0.0003
  - ppo_gamma: 0.995
  - ppo_lambda: 0.97
Registering ATENAcont-v0 environment
✅ Using Snorkel compatibility adapter
Configuration loaded with:
  - humanity_coeff: 1.0
  - diversity_coeff: 2.0
  - kl_coeff: 1.5
  - compaction_coeff: 2.0
  - adam_lr: 0.0003
  - ppo_gamma: 0.995
  - ppo_lambda: 0.97
✅ REWARD STABILIZER: DISABLED (stable mode like train_ipdate-1009-18:54.png)


# Evaluate Human Sessions

This notebook evaluates human analyst exploration sessions to understand how humans explore data.


## Load Human Sessions


In [2]:
%%capture
repo = Repository(
    'gym_atena/reactida/session_repositories/actions_atena.tsv',
    'gym_atena/reactida/session_repositories/displays.tsv',
    'gym_atena/reactida/raw_datasets/'
)

# Filter solutions only
solutions_df, solutions_ids = get_solution_sessions_dfs_and_id(repo)

# Create environment
env = gym.make(env_d)
env.reset()

print(f"Loaded {len(solutions_ids)} human sessions")


## Validate Loading


In [3]:
print(f"Total solution sessions: {len(solutions_ids)}")
print(f"\nFirst 10 actions:")
display(repo.actions.head(10))


Total solution sessions: 120

First 10 actions:


,action_id,action_type,action_params,session_id,user_id,project_id,creation_time,parent_display_id,child_display_id,solution
0,1,group,"{'field': 'eth_src', 'aggregations': [], 'grou...",1,1,1,2016-08-14 12:44:05,1,2,True
1,2,group,"{'field': 'ip_src', 'aggregations': [], 'group...",1,1,1,2016-08-14 12:44:08,2,3,True
2,3,group,"{'field': 'eth_src', 'aggregations': [{'field'...",2,5,1,2016-08-15 09:40:42,4,5,True
3,0,back,{},2,5,1,,5,4,True
4,4,group,"{'field': 'ip_src', 'aggregations': [{'field':...",2,5,1,2016-08-15 13:13:54,4,6,True
5,0,back,{},2,5,1,,6,4,True
6,5,group,"{'field': 'ip_src', 'aggregations': [], 'group...",2,5,1,2016-08-15 13:14:10,4,7,True
7,6,filter,"{'field': 'info_line', 'term': 'reply', 'condi...",2,5,1,2016-08-15 13:15:13,7,8,True
8,0,back,{},2,5,1,,8,7,True
9,8,filter,"{'field': 'highest_layer', 'term': 'ICMP', 'co...",2,5,1,2016-08-15 13:16:34,7,10,True


## Analyze Individual Human Session

Pick a session and analyze it in detail.


In [4]:
# Choose a session to analyze
sol_id = list(solutions_ids)[0]
print(f"Analyzing session {sol_id}")

# Get session details
action_vecs, actions_lst, dataset_id, num_steps = get_solution_details(
    sol_id, solutions_df, verbose=True
)

print(f"\nDataset: {dataset_id}")
print(f"Number of steps: {num_steps}")
print(f"\nActions: {actions_lst}")


Analyzing session 1
Dataset id is 0
{'field': 'eth_src', 'aggregations': [], 'groupPriority': 0}
{'field': 'ip_src', 'aggregations': [], 'groupPriority': 1}

Dataset: 0
Number of steps: 2

Actions: ['group', 'group']


## Run Human Session


In [5]:
# Run the human session
info_hist, total_reward = run_human_session(env, action_vecs, dataset_id, verbose=True)

print(f"\nTotal Reward: {total_reward:.2f}")
print(f"Average Reward per Step: {total_reward / len(info_hist):.2f}")


[2 2 0 0 0 0]
🔧 Rule-based humanity coefficient applied: 1.0
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.659, std=0.000
🔧 Diversity coefficient applied (main calc): 2.0
🔧 Compaction coefficient applied: 2.0
🎯 MASTER-EXACT REWARD:
   action_type: 2
   kl_distance: 0
   compaction_gain: 1.9823869540707095
   total_interestingness: 1.9823869540707095
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.659, std=0.000
🔧 BEFORE coeff: r_snorkel_humanity=0.317988
🔧 Using current humanity_coeff: 1.0
🔧 AFTER coeff: r_snorkel_humanity=0.317988
🎯 Snorkel humanity: 0.317988


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


"Group on Column 'eth_src' and aggregate with '<built-in function len>' on the column 'packet_number'"

[2 6 0 0 0 0]
🔧 Rule-based humanity coefficient applied: 1.0
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.253, std=0.000
🔧 Diversity coefficient applied (main calc): 2.0
🔧 Compaction coefficient applied: 2.0
🎯 MASTER-EXACT REWARD:
   action_type: 2
   kl_distance: 0
   compaction_gain: 0.14675573402404612
   total_interestingness: 0.14675573402404612
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.253, std=0.000
🔧 BEFORE coeff: r_snorkel_humanity=-0.494521
🔧 Using current humanity_coeff: 1.0
🔧 AFTER coeff: r_snorkel_humanity=-0.494521
🎯 Snorkel humanity: -0.494521


"Group on Column 'ip_src' and aggregate with '<built-in function len>' on the column 'packet_number'"


Total Reward: 10.04
Average Reward per Step: 5.02


## Display Session Details


In [6]:
# Display actions and rewards
simulate(info_hist, displays=True, verbose=True)


[2, 2, 0, -0.5, 0, 0],
[2, 6, 0, -0.5, 0, 0],
action: Group on Column 'eth_src' and aggregate with '<built-in function len>' on the column 'packet_number' , reward: 7.988250134417772
raw action: [2, 2, 0, -0.5, 0, 0]
dict_items([('empty_display', 0), ('empty_groupings', 0), ('same_display_seen_already', 0), ('back', 0), ('diversity', 2.0), ('interestingness', 1.9823869540707095), ('kl_distance', 0), ('compaction_gain', 1.9823869540707095), ('humanity', 1.2429881131381761), ('snorkel_humanity', 0.3179881131381761), ('rules_based_humanity', 0.4625), ('human_sessions_similarity', 0), ('coherency_score', 0), ('readability_score', 0), ('diversity_components', {'display_distance': 1.0, 'same_display_penalty': False, 'stabilizer_disabled': True}), ('interestingness_components', {'kl_divergence': 0, 'compaction_gain': 1.9823869540707095, 'information_gain': 0})])



,packet_number
eth_src,
00:0c:29:54:bf:79,7834
00:26:b9:2b:0b:59,814


action: Group on Column 'ip_src' and aggregate with '<built-in function len>' on the column 'packet_number' , reward: 2.05446897946807
raw action: [2, 6, 0, -0.5, 0, 0]
dict_items([('empty_display', 0), ('empty_groupings', 0), ('same_display_seen_already', 0), ('back', 0), ('diversity', 2.0), ('interestingness', 0.14675573402404612), ('kl_distance', 0), ('compaction_gain', 0.14675573402404612), ('humanity', 0.005478755709989014), ('snorkel_humanity', -0.494521244290011), ('rules_based_humanity', 0.25), ('human_sessions_similarity', 0), ('coherency_score', 0), ('readability_score', 0), ('diversity_components', {'display_distance': 1.0, 'same_display_penalty': False, 'stabilizer_disabled': True}), ('interestingness_components', {'kl_divergence': 0, 'compaction_gain': 0.14675573402404612, 'information_gain': 0})])



packet_number
eth_src           ip_src                       
00:0c:29:54:bf:79 0.0.0.0                     2
                  192.168.1.122            7832
00:26:b9:2b:0b:59 0.0.0.0                     2
                  212.134.198.29              2
                  217.206.63.73               2
...                                         ...
                  82.109.10.71                2
                  82.109.65.86                2
                  87.86.75.181                2
                  87.86.75.189                6
                  89.200.130.251              4

[182 rows x 1 columns]

Total Reward: 10.042719113885841


## Analyze Rewards


## Evaluate Multiple Human Sessions

Analyze all human sessions to get aggregate statistics.
ep

In [7]:
# Evaluate multiple sessions (limit for speed)
max_sessions = 10
results = []

for idx, sol_id in enumerate(list(solutions_ids)[:max_sessions]):
    print(f"\nEvaluating session {idx + 1}/{max_sessions} (ID: {sol_id})")
    
    try:
        # Get session details
        action_vecs, actions_lst, dataset_id, num_steps = get_solution_details(
            sol_id, solutions_df, verbose=False
        )
        
        # Run session
        info_hist, total_reward = run_human_session(
            env, action_vecs, dataset_id, verbose=False
        )
        
        results.append({
            'session_id': sol_id,
            'dataset': dataset_id,
            'num_steps': num_steps,
            'total_reward': total_reward,
            'avg_reward': total_reward / num_steps if num_steps > 0 else 0
        })
        
        print(f"  Steps: {num_steps}, Reward: {total_reward:.2f}")
        
    except Exception as e:
        print(f"  Error: {e}")
        continue

# Display results
results_df = pd.DataFrame(results)
print(f"\n\nSummary of {len(results)} human sessions:")
display(results_df)

print(f"\n\nAggregate Statistics:")
print(f"Average steps per session: {results_df['num_steps'].mean():.2f}")
print(f"Average total reward: {results_df['total_reward'].mean():.2f}")
print(f"Average reward per step: {results_df['avg_reward'].mean():.2f}")



Evaluating session 1/10 (ID: 1)
🔧 Rule-based humanity coefficient applied: 1.0
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.659, std=0.000
🔧 Diversity coefficient applied (main calc): 2.0
🔧 Compaction coefficient applied: 2.0
🎯 MASTER-EXACT REWARD:
   action_type: 2
   kl_distance: 0
   compaction_gain: 1.9823869540707095
   total_interestingness: 1.9823869540707095
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.659, std=0.000
🔧 BEFORE coeff: r_snorkel_humanity=0.317988
🔧 Using current humanity_coeff: 1.0
🔧 AFTER coeff: r_snorkel_humanity=0.317988
🎯 Snorkel humanity: 0.317988
🔧 Rule-based humanity coefficient applied: 1.0
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.253, std=0.000
🔧 Diversity coefficient applied (main calc): 2.0
🔧 Compaction coefficient applied: 2.0
🎯 MASTER-EXACT REWARD:
   action_type: 2
   kl_distance: 0
   compaction_gain: 0

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the 

🔧 Rule-based humanity coefficient applied: 1.0
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.192, std=0.000
🔧 Diversity coefficient applied (main calc): 2.0
🔧 Compaction coefficient applied: 2.0
🎯 MASTER-EXACT REWARD:
   action_type: 2
   kl_distance: 0
   compaction_gain: 0.2923061463621609
   total_interestingness: 0.2923061463621609
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.192, std=0.000
🔧 BEFORE coeff: r_snorkel_humanity=-0.616566
🔧 Using current humanity_coeff: 1.0
🔧 AFTER coeff: r_snorkel_humanity=-0.616566
🎯 Snorkel humanity: -0.616566
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.225, std=0.000
🔄 BACK ACTION: humanity rewards only
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.225, std=0.000
🔧 BEFORE coeff: r_snorkel_humanity=-0.549434
🔧 Using current humanity_coeff: 1.0
🔧 AFTER coeff: r_s

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


✅ Real Snorkel predictions: mean=0.659, std=0.000
🔧 BEFORE coeff: r_snorkel_humanity=0.317988
🔧 Using current humanity_coeff: 1.0
🔧 AFTER coeff: r_snorkel_humanity=0.317988
🎯 Snorkel humanity: 0.317988
🔧 Rule-based humanity coefficient applied: 1.0
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.253, std=0.000
🔧 Diversity coefficient applied (main calc): 2.0
🔧 Compaction coefficient applied: 2.0
🎯 MASTER-EXACT REWARD:
   action_type: 2
   kl_distance: 0
   compaction_gain: 0.14675573402404612
   total_interestingness: 0.14675573402404612
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.253, std=0.000
🔧 BEFORE coeff: r_snorkel_humanity=-0.494521
🔧 Using current humanity_coeff: 1.0
🔧 AFTER coeff: r_snorkel_humanity=-0.494521
🎯 Snorkel humanity: -0.494521
🔧 Rule-based humanity coefficient applied: 1.0
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.342, std

...

🔧 KL coefficient applied: 1.5
🎯 MASTER-EXACT REWARD:
   action_type: 1
   kl_distance: 0.13573235357509303
   compaction_gain: 0
   total_interestingness: 0.13573235357509303
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.500, std=0.000
🔧 BEFORE coeff: r_snorkel_humanity=0.000000
🔧 Using current humanity_coeff: 1.0
🔧 AFTER coeff: r_snorkel_humanity=0.000000
🎯 Snorkel humanity: 0.000000
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.336, std=0.000
🔄 BACK ACTION: humanity rewards only
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.336, std=0.000
🔧 BEFORE coeff: r_snorkel_humanity=-0.328848
🔧 Using current humanity_coeff: 1.0
🔧 AFTER coeff: r_snorkel_humanity=-0.328848
🎯 Snorkel humanity: -0.328848
🔧 Rule-based humanity coefficient applied: 1.0
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.500, std=0.000
🔧 

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


✅ Real Snorkel predictions: mean=0.192, std=0.000
🔧 Diversity coefficient applied (main calc): 2.0
🔧 Compaction coefficient applied: 2.0
🎯 MASTER-EXACT REWARD:
   action_type: 2
   kl_distance: 0
   compaction_gain: 0.2923061463621609
   total_interestingness: 0.2923061463621609
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.192, std=0.000
🔧 BEFORE coeff: r_snorkel_humanity=-0.616566
🔧 Using current humanity_coeff: 1.0
🔧 AFTER coeff: r_snorkel_humanity=-0.616566
🎯 Snorkel humanity: -0.616566
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.225, std=0.000
🔄 BACK ACTION: humanity rewards only
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.225, std=0.000
🔧 BEFORE coeff: r_snorkel_humanity=-0.549434
🔧 Using current humanity_coeff: 1.0
🔧 AFTER coeff: r_snorkel_humanity=-0.549434
🎯 Snorkel humanity: -0.549434
🔧 Rule-based humanity coefficient applied: 1.0
🎯 

,session_id,dataset,num_steps,total_reward,avg_reward
0,1,0,2,10.042719,5.021360
1,2,0,21,-15.288646,-0.728031
2,7,0,15,11.900836,0.793389
3,9,1,8,20.308321,2.538540
4,11,0,6,12.072530,2.012088
5,12,1,8,11.810963,1.476370
6,17,0,16,24.063136,1.503946
7,23,0,17,15.714120,0.924360
8,33,0,17,10.473361,0.616080
9,49,0,3,3.328260,1.109420




Aggregate Statistics:
Average steps per session: 11.30
Average total reward: 10.44
Average reward per step: 1.53
